## Quick introduction to generating counterfactual explanations using DiCE

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

DiCE requires two inputs: a training dataset and a pre-trained ML model. It can also work without access to the full dataset (see this [notebook](DiCE_with_private_data.ipynb) for advanced examples).

### Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in **dice_ml.utils.helpers** module. 

In [2]:
dataset = helpers.load_adult_income_dataset()

This dataset has 8 features. The outcome is income which is binarized to 0 (low-income, <=50K) or 1 (high-income, >50K). 

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

Given this dataset, we construct a data object for DiCE. Since continuous and discrete features have different ways of perturbation, we need to specify the names of the continuous features. DiCE also requires the name of the output variable that the ML model will predict.

In [5]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### Loading the ML model

Below, we use a pre-trained ML model which produces high accuracy comparable to other baselines. For convenience, we include the sample trained model with the DiCE package.

In [6]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable

import numpy as np

from dice_ml.model_interfaces.pytorch_model import PyTorchModel

In [7]:
inp_shape= len(d.encoded_feature_names)

ML_modelpath = helpers.get_adult_income_modelpath()
m = PyTorchModel(inp_shape)

learning_rate = 0.001
# Default Batch Size of Keras
batch_size = 32
optimizer = optim.Adam([
    {'params': filter(lambda p: p.requires_grad, m.ann_model.parameters()) }
], lr=learning_rate)
crieterion= nn.CrossEntropyLoss()

In [9]:
#Pre Trained
base_model_dir= '../dice_ml/utils/sample_trained_models/'
dataset_name= 'adult'
path=base_model_dir+dataset_name+'-pytorch.pth'
m.load_state_dict(torch.load(path))
m.eval()

PyTorchModel(
  (ann_model): Sequential(
    (0): Linear(in_features=29, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=2, bias=True)
    (3): Softmax(dim=None)
  )
)

In [11]:
# Dataset for training Black Box Model
train_data_vae= d.data_df.copy()

#Creating list of encoded categorical and continuous feature indices
encoded_categorical_feature_indexes = d.get_data_params()[2]     
encoded_continuous_feature_indexes=[]
data_size= len(d.encoded_feature_names)
for i in range(data_size):
    valid=1
    for v in encoded_categorical_feature_indexes:
        if i in v:
            valid=0
    if valid:
        encoded_continuous_feature_indexes.append(i)            
encoded_start_cat = len(encoded_continuous_feature_indexes)
        
#One Hot Encoding for categorical features
encoded_data = d.one_hot_encode_data(train_data_vae)

# The output/outcome variable position altered due to one_hot_encoding for categorical features: (Cont feat, Outcome, Cat feat) 
# Need to rearrange columns such that outcome variable comes at the last
cols = list(encoded_data.columns)
cols = cols[:encoded_start_cat] + cols[encoded_start_cat+1:] + [cols[encoded_start_cat]]
encoded_data = encoded_data[cols]     

#Normlization for conitnuous features
encoded_data= d.normalize_data(encoded_data)
print(encoded_data.columns)
dataset = encoded_data.to_numpy()

#Train, Val, Test Splits
np.random.shuffle(dataset)
test_size= int(0.2*dataset.shape[0])
val_dataset= dataset[:test_size]
train_dataset= dataset[test_size:]

Index(['age', 'hours_per_week', 'workclass_Government',
       'workclass_Other/Unknown', 'workclass_Private',
       'workclass_Self-Employed', 'education_Assoc', 'education_Bachelors',
       'education_Doctorate', 'education_HS-grad', 'education_Masters',
       'education_Prof-school', 'education_School', 'education_Some-college',
       'marital_status_Divorced', 'marital_status_Married',
       'marital_status_Separated', 'marital_status_Single',
       'marital_status_Widowed', 'occupation_Blue-Collar',
       'occupation_Other/Unknown', 'occupation_Professional',
       'occupation_Sales', 'occupation_Service', 'occupation_White-Collar',
       'race_Other', 'race_White', 'gender_Female', 'gender_Male', 'income'],
      dtype='object')


In [ ]:
#Training
for epoch in range(50):
    np.random.shuffle(train_dataset)
    train_batches= np.array_split( train_dataset, train_dataset.shape[0]//batch_size ,axis=0 )    
    print('Epoch: ', epoch)
    train_acc=0.0
    for i in range(len(train_batches)):    
        optimizer.zero_grad()
        train_x= torch.tensor( train_batches[i][:,:-1] ).float() 
        train_y= torch.tensor( train_batches[i][:,-1], dtype=torch.int64 )
        
        out= m(train_x)
        train_acc += torch.sum( torch.argmax(out, axis=1) == train_y )
        
        # Cross Entropy Loss
        loss= crieterion(out, train_y)
        #L2 Regularization
        weight_norm = torch.tensor(0.)
        for w in m.ann_model.parameters():
            weight_norm += w.norm().pow(1)
        loss+= 0.001*weight_norm
        
        loss.backward()
        optimizer.step()
    print(train_acc, len(train_dataset))     

In [12]:
# Validation        
np.random.shuffle(val_dataset)
train_batches= np.array_split( val_dataset, val_dataset.shape[0]//batch_size ,axis=0 )    
val_acc=0.0
for i in range(len(train_batches)):    
    optimizer.zero_grad()
    train_x= torch.tensor( train_batches[i][:,:-1] ).float() 
    train_y= torch.tensor( train_batches[i][:,-1], dtype=torch.int64 )
    out= m(train_x)
    val_acc += torch.sum( torch.argmax(out, axis=1) == train_y )
print(val_acc, len(val_dataset))	

#Saving the Black Box Model
base_model_dir= '../dice_ml/utils/sample_trained_models/'
dataset_name= 'adult'
path=base_model_dir+dataset_name+'-pytorch.pth'
torch.save(m.state_dict(), path)   

tensor(5426.) 6512


/mnt/c/Users/t-dimaha/Workspace/Dice/env/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


For an example of how to train your own model, check out [this](DiCE_with_advanced_options.ipynb) notebook. 

### Generate feasible counterfactuals

Based on the data object *d* and the model object *m*, we can now instantiate the DiCE class for generating explanations. 

In [13]:
from dice_ml.dice_interfaces.dice_base_gencf import DiceBaseGenCF

In [14]:
# initiate DiCE
exp = DiceBaseGenCF(d, m)
exp.train(pre_trained=1)

/mnt/c/Users/t-dimaha/Workspace/Dice/env/lib/python3.6/site-packages/dice_ml-0.2-py3.6.egg/dice_ml/utils/sample_architecture/vae_model.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c=torch.tensor(c).float()
/mnt/c/Users/t-dimaha/Workspace/Dice/env/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Avg wrong cont dim:  tensor(0.)
recon:  tensor(61.0284, grad_fn=<NegBackward>)  KL:  tensor(0.1315, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6304], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(56.9828, grad_fn=<NegBackward>)  KL:  tensor(0.1080, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6293], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(52.2454, grad_fn=<NegBackward>)  KL:  tensor(0.0877, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6279], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(48.2284, grad_fn=<NegBackward>)  KL:  tensor(0.0775, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6253], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(45.8235, grad_fn=<NegBackward>)  KL:  tensor(0.0707, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6231], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(41.9934, grad_fn=<NegBackward>)  KL:  tensor(0.0682, grad_fn=<M

Avg wrong cont dim:  tensor(0.)
recon:  tensor(22.5179, grad_fn=<NegBackward>)  KL:  tensor(0.6219, grad_fn=<MeanBackward0>)  Validity:  tensor([3.5529], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(22.0081, grad_fn=<NegBackward>)  KL:  tensor(0.6425, grad_fn=<MeanBackward0>)  Validity:  tensor([3.4682], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(22.0583, grad_fn=<NegBackward>)  KL:  tensor(0.6756, grad_fn=<MeanBackward0>)  Validity:  tensor([2.9941], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(22.0277, grad_fn=<NegBackward>)  KL:  tensor(0.6878, grad_fn=<MeanBackward0>)  Validity:  tensor([2.5693], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(22.2682, grad_fn=<NegBackward>)  KL:  tensor(0.7053, grad_fn=<MeanBackward0>)  Validity:  tensor([2.2166], grad_fn=<NegBackward>)
Train Avg Loss:  tensor([2.5190], grad_fn=<DivBackward0>) 19745
Avg wrong cont dim:  tensor(0.)
recon:  tensor(21.8

Train Avg Loss:  tensor([1.9023], grad_fn=<DivBackward0>) 19745
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.6174, grad_fn=<NegBackward>)  KL:  tensor(0.6072, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0496], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.4403, grad_fn=<NegBackward>)  KL:  tensor(0.6231, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0525], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.0458, grad_fn=<NegBackward>)  KL:  tensor(0.6184, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0542], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.0619, grad_fn=<NegBackward>)  KL:  tensor(0.6342, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0481], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.4890, grad_fn=<NegBackward>)  KL:  tensor(0.6336, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0526], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.2

Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.8711, grad_fn=<NegBackward>)  KL:  tensor(0.6858, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0511], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.8396, grad_fn=<NegBackward>)  KL:  tensor(0.6917, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0703], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.9594, grad_fn=<NegBackward>)  KL:  tensor(0.6670, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0527], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.9695, grad_fn=<NegBackward>)  KL:  tensor(0.6589, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0515], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.2763, grad_fn=<NegBackward>)  KL:  tensor(0.6239, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0390], grad_fn=<NegBackward>)
Train Avg Loss:  tensor([1.7939], grad_fn=<DivBackward0>) 19745
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.4

Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.4018, grad_fn=<NegBackward>)  KL:  tensor(0.6274, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0917], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.5424, grad_fn=<NegBackward>)  KL:  tensor(0.5970, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0900], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.4631, grad_fn=<NegBackward>)  KL:  tensor(0.5851, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0548], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.5063, grad_fn=<NegBackward>)  KL:  tensor(0.5883, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0794], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.3314, grad_fn=<NegBackward>)  KL:  tensor(0.6153, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0737], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.4491, grad_fn=<NegBackward>)  KL:  tensor(0.6093, grad_fn=<MeanBa

Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.2310, grad_fn=<NegBackward>)  KL:  tensor(0.6039, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1068], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.8417, grad_fn=<NegBackward>)  KL:  tensor(0.6002, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0860], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.8776, grad_fn=<NegBackward>)  KL:  tensor(0.6017, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0741], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.9014, grad_fn=<NegBackward>)  KL:  tensor(0.6060, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0795], grad_fn=<NegBackward>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.0809, grad_fn=<NegBackward>)  KL:  tensor(0.6157, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0935], grad_fn=<NegBackward>)
Train Avg Loss:  tensor([1.6790], grad_fn=<DivBackward0>) 19745
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.9

DiCE is a form of a local explanation and requires an query input whose outcome needs to be explained. Below we provide a sample input whose outcome is 0 (low-income) as per the ML model object *m*. 

In [15]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

Given the query input, we can now generate counterfactual explanations to show perturbed inputs from the original input where the ML model outputs class 1 (high-income). 

In [16]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=15, desired_class="opposite")

1


/mnt/c/Users/t-dimaha/Workspace/Dice/env/lib/python3.6/site-packages/dice_ml-0.2-py3.6.egg/dice_ml/utils/sample_architecture/vae_model.py:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c=torch.tensor(c).float()


In [17]:
# visualize the results
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0


Please specify a valid posthoc_sparsity_param to perform sparsity correction.. displaying without posthoc sparsity operation


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
1,23.0,Private,Doctorate,Married,White-Collar,White,Male,37.0,1
2,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
3,23.0,Private,Doctorate,Married,White-Collar,White,Male,38.0,1
4,26.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
5,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
6,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
7,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
8,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
9,31.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1


That's it! You can try generating counterfactual explanations for other examples using the same code. You can also try out a use-case for sensitive data in [this](DiCE_with_private_data.ipynb) notebook. 

If you are curious about changing the number of explanations showns, feasibility of the explanations, or how to weigh different features for perturbing, check out how to change DiCE's behavior in this [advanced notebook](DiCE_with_advanced_options.ipynb).  

The counterfactuals generated above are slightly different from those shown in [our paper](https://arxiv.org/pdf/1905.07697.pdf), where the loss convergence condition was made more conservative for rigorous experimentation. To replicate the results in the paper, add an argument *loss_converge_maxiter=2* (the default value is 1) in the *exp.generate_counterfactuals()* method above. For more info, see *generate_counterfactuals()* method in [dice_ml.dice_interfaces.dice_tensorflow.py](../dice_ml/dice_interfaces/dice_tensorflow.py).

###

In [19]:
from dice_ml.dice_interfaces.dice_model_approx_gencf import DiceModelApproxGenCF

In [23]:
# initiate DiCE
exp = DiceModelApproxGenCF(d, m)
exp.train(1, [[0]], 1, 10, pre_trained=1)

Avg wrong cont dim:  tensor(0.)
recon:  tensor(51.5736, grad_fn=<NegBackward>)  KL:  tensor(0.1322, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6328], grad_fn=<NegBackward>)
Constraint:  tensor(0.2281, grad_fn=<MulBackward0>) tensor(0.2281, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(47.2514, grad_fn=<NegBackward>)  KL:  tensor(0.1149, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6325], grad_fn=<NegBackward>)
Constraint:  tensor(0.2992, grad_fn=<MulBackward0>) tensor(0.2992, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(44.4376, grad_fn=<NegBackward>)  KL:  tensor(0.1051, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6322], grad_fn=<NegBackward>)
Constraint:  tensor(0.3287, grad_fn=<MulBackward0>) tensor(0.3287, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(42.7101, grad_fn=<NegBackward>)  KL:  tensor(0.0923, grad_fn=<MeanBackward0>)  Validity:  tensor([16.6316], grad_fn=<NegBackward>)
Constraint: 

Train Avg Loss:  tensor([2.8721], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(21.8167, grad_fn=<NegBackward>)  KL:  tensor(0.8094, grad_fn=<MeanBackward0>)  Validity:  tensor([4.3501], grad_fn=<NegBackward>)
Constraint:  tensor(0.3318, grad_fn=<MulBackward0>) tensor(0.3318, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(21.8678, grad_fn=<NegBackward>)  KL:  tensor(0.8405, grad_fn=<MeanBackward0>)  Validity:  tensor([3.5065], grad_fn=<NegBackward>)
Constraint:  tensor(0.2932, grad_fn=<MulBackward0>) tensor(0.2932, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(21.6576, grad_fn=<NegBackward>)  KL:  tensor(0.8529, grad_fn=<MeanBackward0>)  Validity:  tensor([2.8469], grad_fn=<NegBackward>)
Constraint:  tensor(0.2841, grad_fn=<MulBackward0>) tensor(0.2841, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(21.8529, grad_fn=<NegBackward>)  KL:  tensor(0.8517, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([2.0258], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.7168, grad_fn=<NegBackward>)  KL:  tensor(0.6876, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1863], grad_fn=<NegBackward>)
Constraint:  tensor(0.3694, grad_fn=<MulBackward0>) tensor(0.3694, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.5655, grad_fn=<NegBackward>)  KL:  tensor(0.7074, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1795], grad_fn=<NegBackward>)
Constraint:  tensor(0.2985, grad_fn=<MulBackward0>) tensor(0.2985, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.5713, grad_fn=<NegBackward>)  KL:  tensor(0.7068, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1825], grad_fn=<NegBackward>)
Constraint:  tensor(0.2997, grad_fn=<MulBackward0>) tensor(0.2997, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(18.6727, grad_fn=<NegBackward>)  KL:  tensor(0.7209, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.8617], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.5306, grad_fn=<NegBackward>)  KL:  tensor(0.6963, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0877], grad_fn=<NegBackward>)
Constraint:  tensor(0.2584, grad_fn=<MulBackward0>) tensor(0.2584, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.5115, grad_fn=<NegBackward>)  KL:  tensor(0.6909, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0834], grad_fn=<NegBackward>)
Constraint:  tensor(0.3199, grad_fn=<MulBackward0>) tensor(0.3199, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.3533, grad_fn=<NegBackward>)  KL:  tensor(0.6856, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0827], grad_fn=<NegBackward>)
Constraint:  tensor(0.3219, grad_fn=<MulBackward0>) tensor(0.3219, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.5705, grad_fn=<NegBackward>)  KL:  tensor(0.6964, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.7953], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.7308, grad_fn=<NegBackward>)  KL:  tensor(0.7075, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0562], grad_fn=<NegBackward>)
Constraint:  tensor(0.2507, grad_fn=<MulBackward0>) tensor(0.2507, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.8943, grad_fn=<NegBackward>)  KL:  tensor(0.6818, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0527], grad_fn=<NegBackward>)
Constraint:  tensor(0.2808, grad_fn=<MulBackward0>) tensor(0.2808, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.6077, grad_fn=<NegBackward>)  KL:  tensor(0.6600, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0554], grad_fn=<NegBackward>)
Constraint:  tensor(0.2860, grad_fn=<MulBackward0>) tensor(0.2860, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(17.1109, grad_fn=<NegBackward>)  KL:  tensor(0.6792, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.7725], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.3743, grad_fn=<NegBackward>)  KL:  tensor(0.6773, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0710], grad_fn=<NegBackward>)
Constraint:  tensor(0.2582, grad_fn=<MulBackward0>) tensor(0.2582, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.8069, grad_fn=<NegBackward>)  KL:  tensor(0.6506, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0795], grad_fn=<NegBackward>)
Constraint:  tensor(0.2997, grad_fn=<MulBackward0>) tensor(0.2997, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.4902, grad_fn=<NegBackward>)  KL:  tensor(0.6586, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0626], grad_fn=<NegBackward>)
Constraint:  tensor(0.2950, grad_fn=<MulBackward0>) tensor(0.2950, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.5669, grad_fn=<NegBackward>)  KL:  tensor(0.6527, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.6706], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.3797, grad_fn=<NegBackward>)  KL:  tensor(0.6781, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0811], grad_fn=<NegBackward>)
Constraint:  tensor(0.2993, grad_fn=<MulBackward0>) tensor(0.2993, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.3803, grad_fn=<NegBackward>)  KL:  tensor(0.6675, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0883], grad_fn=<NegBackward>)
Constraint:  tensor(0.2513, grad_fn=<MulBackward0>) tensor(0.2513, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.9927, grad_fn=<NegBackward>)  KL:  tensor(0.6771, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0823], grad_fn=<NegBackward>)
Constraint:  tensor(0.2528, grad_fn=<MulBackward0>) tensor(0.2528, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.8892, grad_fn=<NegBackward>)  KL:  tensor(0.6814, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.7040], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.0215, grad_fn=<NegBackward>)  KL:  tensor(0.7118, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0888], grad_fn=<NegBackward>)
Constraint:  tensor(0.2335, grad_fn=<MulBackward0>) tensor(0.2335, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.0067, grad_fn=<NegBackward>)  KL:  tensor(0.6798, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1162], grad_fn=<NegBackward>)
Constraint:  tensor(0.3118, grad_fn=<MulBackward0>) tensor(0.3118, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(16.1650, grad_fn=<NegBackward>)  KL:  tensor(0.6634, grad_fn=<MeanBackward0>)  Validity:  tensor([0.1053], grad_fn=<NegBackward>)
Constraint:  tensor(0.2355, grad_fn=<MulBackward0>) tensor(0.2355, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.6540, grad_fn=<NegBackward>)  KL:  tensor(0.6655, grad_fn=<MeanBackward0>)  Val

Train Avg Loss:  tensor([1.6797], grad_fn=<DivBackward0>) 19774
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.8146, grad_fn=<NegBackward>)  KL:  tensor(0.6624, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0798], grad_fn=<NegBackward>)
Constraint:  tensor(0.2221, grad_fn=<MulBackward0>) tensor(0.2221, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.4957, grad_fn=<NegBackward>)  KL:  tensor(0.6757, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0813], grad_fn=<NegBackward>)
Constraint:  tensor(0.2759, grad_fn=<MulBackward0>) tensor(0.2759, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.9650, grad_fn=<NegBackward>)  KL:  tensor(0.6821, grad_fn=<MeanBackward0>)  Validity:  tensor([0.0742], grad_fn=<NegBackward>)
Constraint:  tensor(0.2934, grad_fn=<MulBackward0>) tensor(0.2934, grad_fn=<MeanBackward0>)
Avg wrong cont dim:  tensor(0.)
recon:  tensor(15.7476, grad_fn=<NegBackward>)  KL:  tensor(0.6680, grad_fn=<MeanBackward0>)  Val

In [24]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=15, desired_class="opposite")
# visualize the results
dice_exp.visualize_as_dataframe()

1
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.0


Please specify a valid posthoc_sparsity_param to perform sparsity correction.. displaying without posthoc sparsity operation


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
1,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
2,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
3,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
4,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
5,25.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
6,26.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
7,28.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
8,28.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
9,24.0,Private,Doctorate,Married,White-Collar,White,Male,40.0,1
